In [101]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import time

In [34]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [102]:
class utilities:
    def wait(self, seconds):

        s = np.random.normal(0, 1)
        time.sleep(seconds*abs(s))
    
    def get_element_by_path(self, driver, xpath):
        try:
            return driver.find_element_by_xpath(xpath)
        except Exception as e:
            print(e)
            return None

In [269]:
class IndeedItemParser:
    def __init__(self):
        self.driverPath = "C:\\Users\\Junior\\Documents\\selenium\\driver\\chromedriver.exe"
        
    def _get_title(self, driver):
        try:
            title = driver.find_element_by_xpath("//*[@class='jobsearch-DesktopStickyContainer']//h3")
        except Exception as e:
            print(e)
        return title.text
    
    def _get_name(self, driver):
        
        try:    
            name = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[1]")
            return name.text
        except Exception as e:
            print(e)
        
    
    def _get_address(self,driver):
         try:   
            address = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[3]")
            if address.text == "-":
                address = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[4]")
            return address.text
         except Exception as e:
            address = driver.find_element_by_xpath("//span[@class='jobsearch-JobMetadataHeader-iconLabel'][1]")
            return address.text
    
    def _get_description(self,driver):
        try:
            #jobDescriptionText
            e_description = driver.find_element_by_id("jobDescriptionText")
            return e_description.get_attribute('innerHTML')
        except Exception as e:
            print(e)
    
    def _get_date(self,driver,url,name):
       # print(url)
        try:
            date_str = driver.find_element_by_xpath("//*[@class='jobsearch-JobMetadataFooter']")
            date_str_full = date_str.text
            date_tbl = date_str_full.split(" ")
            count_str = date_tbl[4]
            label = date_tbl[5]
            
            if name in date_str_full:
                date_str_full = date_str_full.replace(name, "")
                date_tbl = date_str_full.split(" ")
                count_str = date_tbl[5]
                label = date_tbl[6]
                
            if count_str == "a" :
                count_str = date_tbl[5]
                label = date_tbl[6]
                
            #print("date_str", date_tbl)
            date = datetime.now()
            
            if count_str == "30+":
                return date - timedelta(days=30)
            
            count = int(count_str)
            if "jour" in label:
                date = date - timedelta(days=count)
            elif "heur" in label:
                date = date - timedelta(hours=count)
            return date;
        except Exception as e:
            print(e)
    
    
    def parse(self,url):
        driver = webdriver.Chrome(self.driverPath)
        driver.get(url)
        driver.maximize_window()
        
        title = self._get_title(driver)
        name = self._get_name(driver)
        address = self._get_address(driver)
        date = self._get_date(driver, url,name)
        description = self._get_description(driver)
        
        #driver.get("data:text/html;charset=utf-8,{html_content}".format(html_content=description))
        #xx = driver.find_elements_by_tag_name("p")
        #print("len(xx) : ",len(xx))
        
        driver.close()
        
        return title, name, address, date, description

In [272]:
from datetime import datetime, timedelta
class indeedPaser:
    def __init__(self):
        self.website = "https://www.indeed.fr"
        self.driverPath = "C:\\Users\\Junior\\Documents\\selenium\\driver\\chromedriver.exe"
        self.dataset = pd.DataFrame(columns=['URL', 'Titre','Nom entreprise','Adresse','Date de publication', 'description'])
        self.jobs = ["développeur", "data scientist", "data analyst", "business intelligence"]
        self.locations = ["Paris", "Lyon", "Toulouse", "Nantes", "Bordeaux"]
        self.utilities =  utilities()
        self.indeed_item_parser = IndeedItemParser()
        
    
    def parse(self):
        browser = webdriver.Chrome(self.driverPath)
        browser.get(self.website)
        browser.maximize_window()
        
        for job in self.jobs:
            for location in self.locations:
                query = "https://www.indeed.fr/jobs?q={0}&l={1}".format(job, location)
                browser.get(query)
        
                #searchCountPages
                searchCountPages_elt = browser.find_element_by_id("searchCountPages")
                searchCountPages = searchCountPages_elt.text.split()
                if len(searchCountPages) == 6:
                    searchCountPages = int("{0}{1}".format(searchCountPages[3],searchCountPages[4])) 
                else :
                    searchCountPages = searchCountPages[3]  
                    
                pages_count = (int)(searchCountPages / 18)
                
                for page_index in range(1, pages_count):
                    browser.get("{0}&start={1}".format(query,page_index))
                    
                    #jobsearch-SerpJobCard unifiedRow row result clickcard
                    items = browser.find_elements_by_xpath("//*[contains(@class,'clickcard')]")
                    dataset_len = len(self.dataset)
                    for index_i,item in enumerate(items): 
                        title = item.find_element_by_xpath(".//*[@class='title']//a")
                        item_link = title.get_attribute("href")
                        
                        if (len(self.dataset) == 0) | (item_link not in self.dataset["URL"]):
                            print("existe pas")
                            title, name, address, date, description = self.indeed_item_parser.parse(item_link)
                            self.dataset.loc[dataset_len + index_i] = [item_link, title, name, address, date,description]
                        else:
                            print("existe déjà")
                        
                        #break
                    #break
                    self.dataset.to_csv("indeed.csv", index=False)
                    #https://www.indeed.fr/jobs?q=developpeur&l=paris&start=10
                    
                print(searchCountPages)
                #break
            #break
           
                
                    
                    
                

In [271]:
item_parser = IndeedItemParser()
result = item_parser.parse("https://www.indeed.fr/voir-emploi?jk=9dc37ec01c9984dd&from=serp&vjs=3")
print(result)

('Développeur front-end', 'Be Free Talent', 'Paris (75)', datetime.datetime(2019, 9, 27, 15, 40, 52, 362973), '<div><p>Emploi</p>\n<p><b>\nFormation :</b> un bac + 2 dans une école d’informatique est largement suffisant pour accéder à ce métier ou une reconversion professionnelle <i>via</i> un centre de formation informatique <b>(formation intensive de 3 à 6 mois)</b> permet d’accéder au métier de développeur front-end ou consultant<i> front-end</i>. Et pour les plus malins, des cours d’<i>openclassrooms</i> sont largement suffisants.</p>\n<p><b>\nActivité :</b> le développeur front-end travaille sur des projets de refonte et/ou création de site web.<br>\nTrès souvent, il dispose des compétences en design qui le permet de réaliser des maquettes du futur site internet.<br>\nIl doit disposer de solides compétences en <b>HTML &amp; CSS et jQuery.</b></p>\n<p><b>\nÉvolution :</b> après quelques expériences professionnelles sur du front-end, le développeur front-end peut monter en compétenc

In [ ]:
test = indeedPaser()
test.parse()

existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
invalid literal for int() with base 10: '-'
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
invalid literal for int() with base 10: '-'
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas
existe pas

In [239]:
test.dataset

,URL,Titre,Nom entreprise,Adresse,Date de publication
0,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Java H/F,Ratp Smart Systems,Paris (75),2019-09-09 14:03:27.601372
2,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Informatique H/F,VALENTE SECURYSTAR,94320 Thiais,2019-10-01 16:03:38.884408
4,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Android (H/F),Universign,Paris (75),2019-09-02 14:03:50.342543
6,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Ionic / Firebase,NEXTT,92200 Neuilly-sur-Seine,2019-09-19 14:04:05.596482
8,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Data Engineer / Développeur NLP H/F,Wolters Kluwer,Paris (75),2019-09-02 14:04:23.332829
10,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Sup3r Développeur Web Full Stack @Startup,GlobeSailor SAS,Paris (75),2019-10-02 13:04:34.307811
12,https://www.indeed.fr/rc/clk?jk=2f40c0f1d86b7b...,Développeur Junior (ReactJS / Node.js),AKOYA,Paris (75),2019-10-01 14:04:46.856326
14,https://www.indeed.fr/company/LEC-Conseil/jobs...,Développeur Fullstack F/H,Tiime,Paris (75),2019-09-27 14:04:59.432521
16,https://www.indeed.fr/rc/clk?jk=b5268c024d6547...,Assistant développeur Front (H/F),OUI.sncf,Paris (75),2019-09-27 14:05:16.700216
18,https://www.indeed.fr/company/Flatlooker/jobs/...,Chargé.e de gestion locative en start-up,Flatlooker,Paris (75),2019-10-01 22:05:27.458715
